<h2>De connectie leggen tussen SSMS en Python</h2>

In [ ]:
import pandas as pd
import pyodbc
import sqlite3

# Connect to SQLite database
conn = sqlite3.connect('..\\..\\sqllite\\go_sales.sqlite')

# Connect to SQL Server
DB = {'servername': 'LAPTOP-HSI9FLRD\\SQLEXPRESS',
      'database': 'DataWarehouseGreatOutdoors'}

export_conn = pyodbc.connect('DRIVER={SQL Server};SERVER=' + DB['servername'] +
                             ';DATABASE=' + DB['database'] + ';Trusted_Connection=yes')

export_cursor = export_conn.cursor()

# Read data from SQLite database
products = pd.read_sql_query("SELECT * FROM product", conn)
product_types = pd.read_sql_query("SELECT * FROM product_type", conn)
product_line = pd.read_sql_query("SELECT * FROM product_line", conn)

# Merge data into one table
combined_product_table = pd.merge(products, product_types, on='PRODUCT_TYPE_CODE')
combined_product_table = pd.merge(combined_product_table, product_line, on='PRODUCT_LINE_CODE')
combined_product_table

# Iterate through rows and insert into SQL Server
for index, row in combined_product_table.iterrows():
    try:
        query = "INSERT INTO Product (PRODUCT_NUMBER, PRODUCT_NAME, PRODUCT_DESCRIPTION, PRODUCT_PRODUCT_LINE_NAME, PRODUCT_PRODUCT_TYPE_NAME, PRODUCT_PRODUCT_COST_AMOUNT, PRODUCT_MARGIN_AMOUNT) VALUES (?, ?, ?, ?, ?, ?, ?)"
        export_cursor.execute(query, row['PRODUCT_NUMBER'], row['PRODUCT_NAME'], row['DESCRIPTION'], row['PRODUCT_LINE_EN'], row['PRODUCT_TYPE_EN'], row['PRODUCTION_COST'], row['MARGIN'])
    except pyodbc.Error as e:
        print(f"Error inserting row {index}: {e}")

export_conn.commit() 
export_conn.close()

Table joins

In [ ]:
# # Read data from SQLite database
# products = pd.read_sql_query("SELECT * FROM sales_staff", conn)
# product_types = pd.read_sql_query("SELECT * FROM sales_branch", conn)
# product_line = pd.read_sql_query("SELECT * FROM country", conn)

# # Merge data into one table
# combined_product_table = pd.merge(products, product_types, on='PRODUCT_TYPE_CODE')
# combined_product_table = pd.merge(combined_product_table, product_line, on='PRODUCT_LINE_CODE')
# combined_product_table

# # Iterate through rows and insert into SQL Server
# for index, row in combined_product_table.iterrows():
#     try:
#         query = "INSERT INTO Product (PRODUCT_NUMBER, PRODUCT_NAME, PRODUCT_DESCRIPTION, PRODUCT_PRODUCT_LINE_NAME, PRODUCT_PRODUCT_TYPE_NAME, PRODUCT_PRODUCT_COST_AMOUNT, PRODUCT_MARGIN_AMOUNT) VALUES (?, ?, ?, ?, ?, ?, ?)"
#         export_cursor.execute(query, row['PRODUCT_NUMBER'], row['PRODUCT_NAME'], row['DESCRIPTION'], row['PRODUCT_LINE_EN'], row['PRODUCT_TYPE_EN'], row['PRODUCTION_COST'], row['MARGIN'])
#     except pyodbc.Error as e:
#         print(f"Error inserting row {index}: {e}") 